In [1]:
from experiments._1_one_user_learn_neighbours.try_some_users import *
from experiments.utils import *
from experiments.datasets import *

Switching to API Credentials #3


In [2]:
uid = 37226353
username = 'Leandro Deyuanini'
s = open_session()
user = s.query(User).get(uid)
X_train, X_test, y_train, y_test = load_or_create_dataframe(uid)

In [3]:
neighbours = get_neighbourhood(uid)

In [4]:
neighbour_ids = [n.id for n in neighbours]

In [5]:
clf = load_model(uid)

In [6]:
evaluate_model(clf, X_train, X_test, y_train, y_test)

Detailed classification report:

Scores on training set.

             precision    recall  f1-score   support

        0.0       0.99      1.00      1.00     79167
        1.0       1.00      0.61      0.76      1267

avg / total       0.99      0.99      0.99     80434

Scores on test set.

             precision    recall  f1-score   support

        0.0       0.99      1.00      1.00     33942
        1.0       0.94      0.60      0.73       530

avg / total       0.99      0.99      0.99     34472



# Análisis de ambigüedades

Este usuario muestra un recall mucho más bajo que los demás.
Esta pérdida de calidad puede deberse a errores forzosos causados por ambigüedades en el dataset de entrenamiento.
Diremos que hay ambigüedad para un vector de features $v$ si $(v,0)$ y $(v,1)$ ocurren ambos una cantidad significativa de veces.

In [7]:
miss_clf_counts, details = count_doomed_samples(X_train, y_train)

In [8]:
details[:30]

[([u'@lauritalonso'], {0.0: 340, 1.0: 106}),
 ([u'Paula Mar\xeda'], {0.0: 282, 1.0: 43}),
 ([u'FestivalesGCBA'], {0.0: 657, 1.0: 30}),
 ([u'Federico Pinedo'], {0.0: 125, 1.0: 26}),
 ([u'Enzo Pagani'], {0.0: 132, 1.0: 23}),
 ([u'Eduardo Macchiavelli'], {0.0: 159, 1.0: 22}),
 ([u'PRO'], {0.0: 114, 1.0: 19}),
 ([u'Gladys Gonzalez'], {0.0: 51, 1.0: 16}),
 ([u'Gabriela Michetti'], {0.0: 23, 1.0: 15}),
 ([u'Victoria Morales G'], {0.0: 61, 1.0: 14}),
 ([u'Jorge Macri'], {0.0: 41, 1.0: 14}),
 ([u'Nestor Grindetti'], {0.0: 27, 1.0: 13}),
 ([u'Julio Garro'], {0.0: 296, 1.0: 13}),
 ([u'Cristian Ritondo'], {0.0: 22, 1.0: 8}),
 ([u'Dami\xe1n Esteban Glanz'], {0.0: 97, 1.0: 7}),
 ([u'Karina Spalla'], {0.0: 12, 1.0: 5}),
 ([u'Esteban Bullrich'], {0.0: 18, 1.0: 5}),
 ([u'Sole Martinez'], {0.0: 9, 1.0: 4}),
 ([u'Oscar Moscariello'], {0.0: 21, 1.0: 4}),
 ([u'Fran Quintana'], {0.0: 14, 1.0: 4}),
 ([u'Fernando de Andreis'], {0.0: 8, 1.0: 4}),
 ([u'beto valdez'], {0.0: 1434, 1.0: 4}),
 ([u'@lauritalonso', 

In [9]:
miss_clf_counts

defaultdict(float, {0: 2.0, 1: 475.0})

Vamos a tratar de mejorar la calidad del modelo agregando usuarios al entorno.
Eligiremos estos usuarios de forma que cumplan dos condiciones:
   * Tengan varios retweets de casos a desambiguar
   * Sean cercanos al usuario central ( Katz )

In [10]:
len(details)

80

In [11]:
# error_sources = [d for d in details if d[1][1.0] >= 10]
# y si tomamos todos?
error_sources = details

unames = [x[0][0] for x in error_sources]
amb_users = {u.username: u for u in neighbours if u.username in unames}
amb_ids = {u.id for u in amb_users.values()}

Obtener tweets ambiguos

In [12]:
sum(X_train.sum(axis=1) == 1)

76696

In [13]:
len(X_train)

80434

In [14]:
# tweets con un solo 1 en todo X_train y con 1 en y_train
cands = X_train[(y_train == 1) & (X_train.sum(axis=1) == 1)]

# y ese en alguno de los amb_users
amb_tweets = []
for twid in cands.index:
    t = s.query(Tweet).get(twid)
    if t.author_id in amb_ids:
        amb_tweets.append(t)

In [15]:
len(amb_tweets)

316

Obtener usuarios con mas retweets ambiguos

In [16]:
t = amb_tweets[0]

In [17]:
np.mean([len(t.users_retweeted) for t in amb_tweets])

1.1455696202531647

In [18]:
amb_rt_counts = defaultdict(int)
for t in amb_tweets:
    for u in t.users_retweeted:
        if u.id not in (neighbour_ids + [user.id]):
            amb_rt_counts[u.id] += 1
    if t.author_id not in (neighbour_ids + [user.id]):
        amb_rt_counts[t.author_id] += 1

In [19]:
new_neighbours = sorted(amb_rt_counts.items(), key=lambda x: -x[1])

In [20]:
new_neighbours

[(195412602, 3),
 (35710956, 2),
 (188376648, 2),
 (283762641, 2),
 (160955228, 2),
 (7909502, 2),
 (153127426, 1),
 (138835591, 1),
 (337631754, 1),
 (181148556, 1),
 (154225689, 1),
 (127996443, 1),
 (196765605, 1),
 (33798438, 1),
 (715185320, 1),
 (72062123, 1),
 (178025902, 1),
 (227020722, 1),
 (15521401, 1),
 (147800890, 1),
 (8519232, 1),
 (126760513, 1),
 (152769475, 1),
 (158116807, 1),
 (57399758, 1),
 (127715027, 1),
 (14636665, 1),
 (57686497, 1),
 (49651047, 1),
 (145709292, 1),
 (186560753, 1),
 (157833202, 1),
 (83043060, 1),
 (143795703, 1),
 (151598585, 1),
 (157227260, 1),
 (168140287, 1)]

In [21]:
[t.text for t in amb_tweets[:20]]

[u'Tucum\xe1n nos abre una enorme esperanza #Democracia #Justicia',
 u'Nancy y sus vecinos, de Villa Tranquila, viven sin cloacas y con el problema latente de las inundaciones. http://t.co/Ff4BvRZIOU',
 u'Hoy, a las 11, @PerdomoVero y el Dr. Jorge Rocco dan un taller acerca del cuidado de la salud, en @legiscaba. Sumate! http://t.co/O6s0WCofis',
 u'"Todos piden que no aflojemos" @mauriciomacri en San Pedro #Jujuy http://t.co/MME0VFJ4Al http://t.co/TbbhrkThlI',
 u'Con @DSalvadorSF y miembros del Frente Cambiemos de Avellaneda firmando nuestro compromiso #VamosJuntos http://t.co/TcCHuSfqVW',
 u'Conectate a @Laoncediez y escuch\xe1 a Dar\xedo Lop\xe9rfido contando todos los detalles sobre el #10FIBA http://t.co/CY8mbziUaD',
 u'Mauricio Macri y Sergio Massa, en una conferencia conjunta convocada por las irregularidade... http://t.co/8d4Hy9yTqx http://t.co/in7zbtFErh',
 u'Cierre de campa\xf1a #Cambiemos en Laboulaye, C\xf3rdoba. Este domingo C\xe9sar Abdala intendente y Mariano Proietti con

Son pocos los usuarios encontrados, vamos a agregarlos a todos


In [22]:
s = open_session()

new_neighbour_ids = [x[0] for x in new_neighbours]

new_neighbours = [s.query(User).get(uid) for uid in new_neighbour_ids]

In [23]:
user = s.query(User).get(uid)

In [24]:
tweet_ids = X_train.index
tweets = [s.query(Tweet).get(twid) for twid in tweet_ids]

X_train_new, _ = extract_features(tweets, new_neighbours, user)

X_train_new = pd.DataFrame(data=X_train_new, index=tweet_ids, columns=new_neighbour_ids)

X_train_extended = pd.concat([X_train,X_train_new],axis=1)

Ahora entrenamos en el dataset extendido

In [ ]:
# weights for class balancing
w1 = sum(y_train)/len(y_train)
w0 = 1 - w1
sample_weights = np.array([w0 if x==0 else w1 for x in y_train])

In [51]:
clf = RandomForestClassifier()     
clf.fit(X_train_extended, y_train, sample_weight=sample_weights)

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_impurity_split=1e-07, min_samples_leaf=1,
            min_samples_split=2, min_weight_fraction_leaf=0.0,
            n_estimators=10, n_jobs=1, oob_score=False, random_state=None,
            verbose=0, warm_start=False)

Evaluamos para ver si hubo alguna mejora

In [27]:
y_true, y_pred = y_train, clf.predict(X_train_extended)

print("Scores on training set.\n")
print(classification_report(y_true, y_pred))

Scores on training set.

             precision    recall  f1-score   support

        0.0       0.99      1.00      1.00     79167
        1.0       1.00      0.62      0.77      1267

avg / total       0.99      0.99      0.99     80434



Mejoró un $1\%$ el *recall* en el training set.

Veamos si esta mejora se traslada al test set.

In [ ]:
test_tweets = [s.query(Tweet).get(twid) for twid in X_test.index]

X_test_new, _ = extract_features(test_tweets, new_neighbours, user)

X_test_new = pd.DataFrame(data=X_test_new, index=X_test.index, columns=new_neighbour_ids)
X_test_extended = pd.concat([X_test,X_test_new],axis=1)

In [53]:
y_true, y_pred = y_test, clf.predict(X_test_extended)
print(classification_report(y_true, y_pred))

             precision    recall  f1-score   support

        0.0       0.99      1.00      1.00     33942
        1.0       0.91      0.60      0.72       530

avg / total       0.99      0.99      0.99     34472



Bajó la calidad en el test set.

Veamos si bajó la ambigüedad y mejoró el *máximo recall potencial*.

In [28]:
miss_clf_counts_new, details_new = count_doomed_samples(X_train_extended, y_train)

In [29]:
details_new[:20]

[([u'@lauritalonso'], {0.0: 318, 1.0: 93}),
 ([u'Paula Mar\xeda'], {0.0: 278, 1.0: 40}),
 ([u'FestivalesGCBA'], {0.0: 656, 1.0: 30}),
 ([u'Federico Pinedo'], {0.0: 122, 1.0: 25}),
 ([u'Enzo Pagani'], {0.0: 132, 1.0: 23}),
 ([u'Eduardo Macchiavelli'], {0.0: 159, 1.0: 21}),
 ([u'PRO'], {0.0: 114, 1.0: 18}),
 ([u'Gladys Gonzalez'], {0.0: 51, 1.0: 16}),
 ([u'Victoria Morales G'], {0.0: 61, 1.0: 14}),
 ([u'Jorge Macri'], {0.0: 41, 1.0: 14}),
 ([u'Nestor Grindetti'], {0.0: 27, 1.0: 13}),
 ([u'Julio Garro'], {0.0: 296, 1.0: 13}),
 ([u'Gabriela Michetti'], {0.0: 23, 1.0: 13}),
 ([u'Cristian Ritondo'], {0.0: 21, 1.0: 8}),
 ([u'Dami\xe1n Esteban Glanz'], {0.0: 97, 1.0: 7}),
 ([u'Karina Spalla'], {0.0: 12, 1.0: 5}),
 ([u'Sole Martinez'], {0.0: 9, 1.0: 4}),
 ([u'Oscar Moscariello'], {0.0: 21, 1.0: 4}),
 ([u'Fran Quintana'], {0.0: 14, 1.0: 4}),
 ([u'Fernando de Andreis'], {0.0: 8, 1.0: 4})]

In [30]:
miss_clf_counts_new

defaultdict(float, {0: 2.0, 1: 444.0})

In [31]:
max_recall_old = 1 - miss_clf_counts[1]*1.0/sum(y_train)
max_recall_new = 1 - miss_clf_counts_new[1]*1.0/sum(y_train)

In [32]:
max_recall_old

0.62509865824782951

In [33]:
max_recall_new

0.64956590370955014

Pareciera haber un $35\%$ de *falsos negativos* irrecuperables. Es decir, tweets que sólo han sido (re)tweeteados por el usuario central, y que no es posible desambiguar agregando otros usuarios a las features.

In [34]:
len(tweets)

80434

In [35]:
own_tweets = [t for t in tweets if t.author_id == user.id]

In [36]:
pos_twids = [t for i, t in enumerate(tweets) if y_train[i]]

In [112]:
len(pos_twids)

1393

In [115]:
lonely_tweets_and_rts = [t for t in pos_twids if t.retweet_count<=1]

In [116]:
len(lonely_tweets_and_rts)

277

In [120]:
lonely_own_tweets = [t for t in lonely_tweets_and_rts if t.author_id == user.id]

In [121]:
len(lonely_own_tweets)

59

In [122]:
len(own_tweets)

60

Parece que no es muy retweeteado este usuario...

In [125]:
lonely_rts = [t for t in lonely_tweets_and_rts if [u.id for u in t.users_retweeted] == [user.id]]

In [126]:
len(lonely_rts)

218

In [117]:
277.0 / 1393

0.19885139985642497

(re)tweets solitarios sólo cubren el $20\%$ de los *true positives*. Asumimos que el restante $13\%$ tiene retweeters fuera de nuestro grafo de muestra.

Inspeccionemos a qué se deben tantos (re)tweets solitarios

In [119]:
[(t.author_id == user.id, t.text) for t in lonely_tweets_and_rts]

[(False, u'FELIZ D\xcdA MAESTROS!! http://t.co/iZOjHCoAEu'),
 (False,
  u'El s\xe1bado comienza junto a Daniel Pedercini en #Retratos / De 6h a 8h en vivo por http://t.co/Gwdarvi6oR http://t.co/JgkdSLuy0y'),
 (False,
  u'Fernando del Priore te acompa\xf1a de 3 a 6 h con @Tangosmadrugada / Entrevistas, efem\xe9rides, tangos y noticias por http://t.co/Sp1K6FWOOR'),
 (False,
  u'\xbfCual es tu programa favorito de nuestro canal? Contestanos con el # del programa'),
 (False, u'En Balcarce hay tazas de Rada Tilly :) http://t.co/cPtzPAcs5J'),
 (False,
  u'El cantor @brianchambouley en #TangoBA "Pero hay una melena" en vivo desde @el25demayoBA http://t.co/9zxeIdF3P3 http://t.co/p7g5V9KJ9f'),
 (False,
  u'El Quinteto de N\xe9stor Marconi en #TangoBA / "Para el recorrido" en vivo desde @el25demayoBA http://t.co/HRnzz1j8nZ http://t.co/yWnX21ODJr'),
 (False,
  u'Feliz cumple al mega genio del universo @matti_quintana \U0001f388'),
 (False,
  u'M\xc1S DE 20000 PERSONAS VAN A EMPEZAR A VIAJAR MEJOR

# Agregar autor como feature

Veamos si agregar el id del autor de cada tweet como feature mejora la calidad.

In [41]:
X_train_author = X_train.copy()

X_train_author['authorid'] = [t.author_id for t in tweets]

clf = RandomForestClassifier()     
clf.fit(X_train_author, y_train, sample_weight=sample_weights)

y_true, y_pred = y_train, clf.predict(X_train_author)

print(classification_report(y_true, y_pred))

Scores on training set.

             precision    recall  f1-score   support

        0.0       0.99      1.00      1.00     79167
        1.0       0.99      0.65      0.79      1267

avg / total       0.99      0.99      0.99     80434



Mejora un $4\%$ respecto del modelo base. Veamos ahora combinando extensión de vecinos y feature de autor.

In [ ]:
X_train_author = X_train_extended.copy()
X_train_author['authorid'] = [t.author_id for t in tweets]

In [49]:
clf = RandomForestClassifier()     
clf.fit(X_train_author, y_train, sample_weight=sample_weights)

y_true, y_pred = y_train, clf.predict(X_train_author)
print(classification_report(y_true, y_pred))

Scores on training set.

             precision    recall  f1-score   support

        0.0       0.99      1.00      1.00     79167
        1.0       0.98      0.67      0.80      1267

avg / total       0.99      0.99      0.99     80434



Parecen ser independientes ambas mejoras, y se acumula un $5\%$ de mejora.
Veamos que esto no sea *overfitting* y se de en el conjunto de *test*.

In [47]:
X_test_author = X_test_extended.copy()
X_test_author['authorid'] = [t.author_id for t in test_tweets]

In [50]:
y_true, y_pred = y_test, clf.predict(X_test_author)
print(classification_report(y_true, y_pred))

Scores on training set.

             precision    recall  f1-score   support

        0.0       0.99      1.00      0.99     33942
        1.0       0.64      0.57      0.60       530

avg / total       0.99      0.99      0.99     34472



Se pierde mucha calidad en el conjunto de test, estamos haciendo *overfitting*